In [1]:
import pandas as pd

# specify path
path = "/home/cdsw/meurit/tests/fixtures/dummy_config"
filepath = f"{path}/interconnectors.csv"

# load file
intercons = pd.read_csv(filepath)
intercons

# add some additional interconnectors
intercons.at[5] = ('interconnector_be_de', 'be', 'de', 600, 1.0, True)
intercons.at[6] = ('interconnector_be_uk', 'be', 'uk', 330, 1.0, True)
intercons.at[7] = ('interconnector_dk_no', 'dk', 'no', 200, 1.0, True)

intercons.at[0, 'in_service'] = False

# show head
intercons

,name,from_region,to_region,p_mw,scaling,in_service
0,interconnector_nl_be,nl,be,700.0,1.0,False
1,interconnector_nl_de,nl,de,500.0,1.0,True
2,interconnector_nl_dk,nl,dk,1400.0,1.0,True
3,interconnector_nl_no,nl,no,250.0,1.0,True
4,interconnector_nl_uk,nl,uk,240.0,1.0,True
5,interconnector_be_de,be,de,600.0,1.0,True
6,interconnector_be_uk,be,uk,330.0,1.0,True
7,interconnector_dk_no,dk,no,200.0,1.0,True


In [2]:
utilization = pd.DataFrame(0.0, index=range(8760), columns=intercons.index)
utilization.at[1] = (1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0)
utilization.at[2] = (1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0)

# head
utilization.head()

,0,1,2,3,4,5,6,7
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
import numpy as np
import pandas as pd

# make random price curves
countries = ['be', 'de', 'dk', 'nl', 'no', 'uk']
prices = [np.random.randint(0, 100, 8760) for _ in range(len(countries))]
prices = pd.DataFrame(prices, index=countries).T

# make testcase at first hour
prices.at[0] = (0, 0, 0, 0, 0, 0)

# show head
prices.head()

,be,de,dk,nl,no,uk
0,0,0,0,0,0,0
1,94,98,89,56,87,15
2,38,64,7,8,97,88
3,73,12,42,45,22,33
4,46,14,70,94,52,20


### To Do
- DataFrame with operational specification production plants (and capacity)
- DataFrame with dispatch percentage of each plant in each country in each hour
- DataFrame with interconnectors and use

Don't forget to use a price of zero as an indicator that the market or interconnector are saturated.

In [4]:
class MarketModel:

    def __init__(self, interconnectors):
        """class initialization"""

        # set dataframe
        self.interconnectors = interconnectors
        
    @property
    def regions(self):
        """included regions"""
        return np.unique(df[['from_region', 'to_region']])
    
    @property
    def capacity_matrix(self):
        """make connection matrix for interconnectors"""
        return self.__make_property_matrix('p_mw')
    
    @property
    def availability_matrix(self):
        return self.__make_property_matrix('scaling')
    
    @property
    def in_service_matrix(self):
        return self.__make_property_matrix('in_service')
        
    def __make_property_matrix(self, key):
        """get property matrix from interconnection dataframe"""
        
        # prepare dataframe headers
        keys = ['from_region', 'to_region']
        index = np.unique(self.cons[keys])

        # get values and fill dataframe
        values = self.cons.set_index(keys)[key].unstack('to_region')
        matrix = pd.DataFrame(data=values, index=index, columns=index)
        
        return matrix
        
    def get_zonal_price_delta(self, prices, from_region, to_region):
        """get price delta between two regions"""
        return prices[from_region].sub(prices[to_region])
    
    def get_exchange_prices(self, prices):
        """get price deltas for each interconnector"""
        
        # subset available interconnectors
        interconns = self.interconnectors[self.interconnectors.in_service]
        
        # make dictonairy
        frm, to = interconns.from_region, interconns.to_region
        mapping = dict(zip(interconns.index, list(zip(frm, to))))
        
        # evaluate price deltas for combinations
        combs, func = mapping.values(), self.get_zonal_price_delta
        deltas = [func(prices, frm, to) for frm, to in combs] 
        
        return pd.concat(deltas, axis=1, keys=mapping.keys())
    
    def get_capacity_availability(self, utilization):
        """determine wheter an interconnector is available to the
        market or is already fully utilized"""
        
        # determine remaining capacity
        available = self.interconnectors.scaling.sub(utilization)
        capacity = model.interconnectors.p_mw.mul(available)
        
        return capacity        
            
    def _lookup_exchange_prices(self, indices, prices):
        """lookup for exchange prices"""
        
        # factorize hourly deltas
        idx, cols = pd.factorize(indices)
        values = prices.reindex(cols, axis=1)

        # lookup values
        values = values.to_numpy()
        values = values[np.arange(len(values)), idx]

        return pd.Series(values, index=prices.index, dtype='float64')
        
    def make_exchange_table(self, prices, utilization):
        """make table with hourly exchange information"""

        # determine prices and capacity
        prices = self.get_exchange_prices(prices)
        capacity = self.get_capacity_availability(utilization)
        
        # drop exchange for saturated horus 
        prices = prices.mul(capacity > 0)
        
        # get highest exchange potential
        conns = prices.abs().idxmax(axis=1)
        df = conns.to_frame(name='interconnector')

        # lookup corresponding exchange prices
        prices = self._lookup_exchange_prices(conns, prices)

        # merge region information with interconnectors
        keys = ['from_region', 'to_region']
        ref = df.merge(self.interconnectors[keys], how='left', 
                       left_on='interconnector', right_index=True)
                
        # assign exchange positions of regions
        df['export'] = np.where(prices >= 0, ref.from_region, ref.to_region)
        df['import'] = np.where(prices >= 0, ref.to_region, ref.from_region)
        
        # assign exchange price
        df['price'] = prices.abs()
        
        return df
    
# initialize model
model = MarketModel(intercons)

# make exchange table
%time model.make_exchange_table(prices, utilization)

CPU times: user 20 ms, sys: 1.94 ms, total: 22 ms
Wall time: 21.1 ms


,interconnector,export,import,price
0,1,nl,de,0.0
1,1,nl,de,0.0
2,7,no,dk,90.0
3,5,be,de,61.0
4,1,nl,de,80.0
...,...,...,...,...
8755,2,nl,dk,83.0
8756,5,de,be,81.0
8757,5,de,be,79.0
8758,4,uk,nl,62.0
